In [1]:
!pip install langchain pymongo -q

In [2]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from pymongo import MongoClient

In [3]:
llm = Ollama(model="llama3")

In [4]:
client = MongoClient('localhost', 27017)
database = client['LangChain_Intento4']
collection = database['lenguajes']

In [5]:
chat_history = []

### Prompts

#### Eres un AI que agenda citas, me debes ir preguntando mi información hasta obtener lo siguiente:
            - nombre
            - email
            - horario de cita
            Una vez que tengas todo, lo retornarás en forma de diccionario de esta forma: 
            'nombre': <nombre>, 'email': <email>, 'cita': <horario de la cita>

#### Eres un AI que agenda citas, me debes ir preguntando mi información hasta obtener lo siguiente:
            - nombre
            - email
            - horario de cita (día de la semana, día del mes, mes y hora), haz esto mensaje tras mensaje
            Una vez que tengas todo, lo retornarás en forma de diccionario: 
            'nombre': <nombre>, 'email': <email>, 'cita': <horario de la cita>. No retornes nada más que eso

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres un AI que agenda citas, me debes ir preguntando mi información hasta obtener lo siguiente:
            - nombre
            - email
            - horario de cita (día de la semana, día del mes, mes y hora), haz esto mensaje tras mensaje.
            
            Solamente cuando hayas obtenido todo, retornarás los datos en forma de diccionario: 
            'nombre': <nombre>, 'email': <email>, 'cita': <horario de la cita en MM/DD/2024 HH:MM>
            No retornes nada extra más que eso, solamente el diccionario, ninguna palabra más.""",
        ),

        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ])

In [7]:
chain = prompt_template | llm

In [8]:
def chat():
    while True:
        pregunta = input("You: ")
        if pregunta == "bye":
            return
        # response = llm.invoke(question
        response = chain.invoke({"input": pregunta, "chat_history": chat_history})
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(SystemMessage(content=response))
        print("-"*50)
        print("AI: " + response)

        #datos_iniciales = response[-1].content
        datos_iniciales = response
        if datos_iniciales.startswith("{"):
            print("---------------out--------------")
            datos_diccionario = eval(datos_iniciales)
            print(type(datos_diccionario))
            print(datos_diccionario)
            user_nombre = datos_diccionario['nombre']
            print(user_nombre)
            user_email = datos_diccionario['email']
            print(user_email)
            user_cita = datos_diccionario['cita']
            print(user_cita)
            print("---------------out--------------")

            confirmacion = "Preguntame si quiero confirmar la cita, si te digo que no, pregúntame qué quiero modificar, lo modificas y vuelves a retornar el diccionario. Si te digo que sí, retorna solamente True, nada más"
            response = chain.invoke({"input": confirmacion, "chat_history": chat_history}) #aqui el chat nos pregunta si confirmamos
            chat_history.append(HumanMessage(content=confirmacion)) #guardar confirmacion
            chat_history.append(SystemMessage(content=response)) # guardar accion IA
            print("AI: " + response)           
            pregunta = input("You: ") #aquí respondemos a si confirmamos o no
            response = chain.invoke({"input": pregunta, "chat_history": chat_history}) #aqui imprime True
            chat_history.append(HumanMessage(content=pregunta)) # guardar nuestra confirmacion
            chat_history.append(SystemMessage(content=response)) # guardar True
            if str(response) == "True":
                print("---------------out--------------")
                collection.insert_one({"name":user_nombre, "email": user_email, "cita": user_cita})
                print("cita agendada")
                print("---------------out--------------")

In [ ]:
chat()

You:  hola


--------------------------------------------------
AI: ¿Cuál es tu nombre?


You:  soy Osvaldo


--------------------------------------------------
AI: ¿Eres Osvaldo? ¡Genial! ¿Tienes correo electrónico?


You:  sí, osvaldo@email.com


...

--------------------------------------------------
AI: Error! No sé qué hacer con dos opciones de meses. Por favor, elige solo uno.

(¿Qué mes prefieres para la cita? Hasta ahora, sabemos que es martes, 28 de ..., pero no sabemos qué mes es)

(Puedes corregir o cancelar la cita si lo deseas)


You:  mayo


--------------------------------------------------
AI: {'nombre': 'Osvaldo', 'email': 'osvaldo@email.com', 'cita': '05/28/2024 00:00'}
---------------out--------------
<class 'dict'>
{'nombre': 'Osvaldo', 'email': 'osvaldo@email.com', 'cita': '05/28/2024 00:00'}
Osvaldo
osvaldo@email.com
05/28/2024 00:00
---------------out--------------
AI: ¡Hola! Como sistema de citas, tengo una pregunta para ti: ¿Quieres confirmar la cita con Osvaldo en martes, 28 de mayo de 2024 a las 12:00? 

(Puedes responder "sí" o "no", según tu preferencia)


You:  si


---------------out--------------
cita agendada
---------------out--------------
